In [55]:
import json
import csv
import numpy as np
import pandas as pd
from datetime import datetime, timedelta, date
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, Pool

pd.set_option('display.max_columns', 500)

ModuleNotFoundError: No module named 'sklearn'

In [2]:
ds = datetime.strptime('2017-01-01', '%Y-%m-%d')
df=pd.read_csv('data/horses_stats.csv')
df['marketTime']=pd.to_datetime(df['marketTime'])
df=df[df['marketTime']>=ds]

In [45]:
rids=df.rid.unique()
np.random.shuffle(rids)
train, test, validate = np.split(rids, [int(.7*len(rids)), int(.85*len(rids))]) 
train_df=df[df['rid'].isin(train)]
test_df=df[df['rid'].isin(test)]
validate_df=df[df['rid'].isin(validate)]

In [51]:
cols_categorical=[col for col in df.columns if '_rank' in col]
cols=[col for col in df.columns if not col in cols_categorical]
cols_numerical=[col for col in cols if ('_avg' in col) or ('_diff' in col)]
cols=[col for col in cols if not col in cols_numerical]
cols_categorical=cols_categorical+['ncond', 'class','isFav']
cols_numerical=cols_numerical+['RPR', 'TR', 'OR', 'win_drift']
cols_descr=['rid', 'marketTime', 'horseName', 'runners','res_win', 'res_place']

In [52]:
train_x=train_df[cols_categorical+cols_numerical]
train_y=train_df['res_win']
test_x=test_df[cols_categorical+cols_numerical]
test_y=test_df['res_win']
validate_x=validate_df[cols_categorical+cols_numerical]
validate_y=validate_df['res_win']
cat_idxs=np.where(train_x.columns.isin(cols_categorical))[0]
eval_dataset = Pool(validate_x, validate_y,cat_idxs)

NameError: name 'Pool' is not defined

In [49]:
clf = CatBoostClassifier(iterations=100, depth=2, learning_rate=0.01, loss_function='Logloss',use_best_model=True,  verbose=False)
clf.fit(train_x, train_y, cat_features=cat_idxs, eval_set=eval_dataset)
print(clf.get_best_iteration(), clf.get_best_score())   

(299281,)

In [38]:
df.columns 

Index(['Unnamed: 0', 'rid', 'marketTime', 'horseName', 'runners', 'ncond',
       'class', 'res_win', 'res_place', 'decimalPrice', 'isFav', 'RPR', 'TR',
       'OR', 'weight', 'win_drift', 'age_diff', 'age_rank',
       'decimalPrice_diff', 'decimalPrice_rank', 'weight_diff', 'weight_rank',
       'RPR_diff', 'RPR_rank', 'TR_diff', 'TR_rank', 'OR_diff', 'OR_rank',
       'win_drift_diff', 'win_drift_rank', 'position_diff', 'position_rank',
       'metric_h_avg', 'res_win_h_avg', 'res_place_h_avg',
       'decimalPrice_mean_h_avg', 'RPR_mean_h_avg', 'TR_mean_h_avg',
       'OR_mean_h_avg', 'win_drift_h_avg', 'position_mean_h_avg',
       'metric_t_avg', 'res_win_t_avg', 'res_place_t_avg',
       'decimalPrice_mean_t_avg', 'RPR_mean_t_avg', 'TR_mean_t_avg',
       'OR_mean_t_avg', 'win_drift_t_avg', 'position_mean_t_avg',
       'metric_j_avg', 'res_win_j_avg', 'res_place_j_avg',
       'decimalPrice_mean_j_avg', 'RPR_mean_j_avg', 'TR_mean_j_avg',
       'OR_mean_j_avg', 'win_drift_j_avg